In [593]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import pyodbc
import pandas_access as pa
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier
from tensorflow import keras
import tensorflow as tf

In [20]:
dataset_close=pd.read_csv('data\hospital_closure\hospital_closure.csv')

In [3]:
dataset_close['year_close_combined'].value_counts(dropna=False).sort_index()

2002     1
2003     1
2004     1
2005     1
2006     3
2007    13
2008    16
2009    17
2010    12
2011    13
2012    25
2013    23
2014    24
2015    18
2016    10
2017     7
Name: year_close_combined, dtype: int64

In [326]:
MDB = 'data\hospital_compare\Copy of hc_dec2016.mdb'
MDB2='data\hospital_compare\Copy of HospitalMay2015.mdb'
DRV = '{Microsoft Access Driver (*.mdb, *.accdb)}'
PWD = 'pw'

con = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB,PWD))
cur = con.cursor()

list(cur.tables())

[('data\\hospital_compare\\Copy of hc_dec2016.mdb', None, 'MSysAccessObjects', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of hc_dec2016.mdb', None, 'MSysAccessXML', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of hc_dec2016.mdb', None, 'MSysACEs', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of hc_dec2016.mdb', None, 'MSysIMEXColumns', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of hc_dec2016.mdb', None, 'MSysIMEXSpecs', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of hc_dec2016.mdb', None, 'MSysNameMap', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of hc_dec2016.mdb', None, 'MSysNavPaneGroupCategories', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of hc_dec2016.mdb', None, 'MSysNavPaneGroups', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of hc_dec2016.mdb', None, 'MSysNavPaneGroupToObjects', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\Copy of hc_dec2016.mdb', None, 'MSysNavPaneObjectIDs', 'SYSTEM 

In [489]:
table_name = '[Medicare Hospital Spending by Claim]'
table_name2='HQI_HOSP_PaymentAndValueOfCare'
# define query
query = "SELECT * FROM {}".format(table_name)
query2 = "SELECT * FROM {}".format(table_name2)

rows = cur.execute(query).fetchall()

In [544]:
row2 = cur.execute(query2).fetchall()
row2=np.array(row2)
dataset0=pd.DataFrame(data=row2[:,[0,4,5,6,7,8,9,12,13]],columns=['1','2','3','4','5','6','7','8','9'])


In [558]:
dataset0

,1,2,3,4,5,6,7,8,9
0,010001,0,707,21804,20744,22932,None,Average mortality and average payment,None
1,010001,0,754,15659,14884,16446,None,Average mortality and average payment,None
2,010001,0,404,15089,14133,16018,None,Average mortality and average payment,None
3,010005,0,53,23171,20404,26226,None,Average mortality and average payment,None
4,010005,0,347,16376,15237,17547,None,Worse mortality and average payment,None
...,...,...,...,...,...,...,...,...,...
14449,670111,0,NaN,NaN,NaN,NaN,19,Not Available,19
14450,670111,0,NaN,NaN,NaN,NaN,19,Not Available,19
14451,670112,0,NaN,NaN,NaN,NaN,5,Not Available,5
14452,670112,0,NaN,NaN,NaN,NaN,5,Not Available,5


In [546]:
dataset0['2']=dataset0['2'].replace({'No Different than the National Average Payment':0,'Number of Cases Too Small':0,'Not Available':0,'Greater than the National Average Payment':1,'Less than the National Average Payment':2})

In [547]:
dataset0['4']=dataset0['4'].str.strip("$")
dataset0['4']=dataset0['4'].str.replace(',', '')
dataset0['5']=dataset0['5'].str.strip("$")
dataset0['5']=dataset0['5'].str.replace(',', '')
dataset0['6']=dataset0['6'].str.strip("$")
dataset0['6']=dataset0['6'].str.replace(',', '')

In [557]:
dataset0['3']=dataset0['3'].replace({'Not Available':np.nan})
dataset0['4']=dataset0['4'].replace({'Not Available':np.nan})
dataset0['5']=dataset0['5'].replace({'Not Available':np.nan})
dataset0['6']=dataset0['6'].replace({'Not Available':np.nan})

In [553]:
dataset0['6'].value_counts(dropna=False)

Not Available    4574
14373               8
13464               8
13714               8
13422               7
                 ... 
22615               1
10591               1
11503               1
15091               1
16884               1
Name: 5, Length: 6163, dtype: int64

In [340]:
arr=np.array(rows)


In [381]:
dataset4=pd.DataFrame(arr[:,[1,5,6,7,8,9,10]],columns=['id','1','2','3','4','5','6'])

In [382]:
dataset4

,id,1,2,3,4,5,6
0,010019,18,31,24,0.10%,0.16%,0.12%
1,010019,1062,1480,1540,6.01%,7.71%,7.52%
2,010019,917,948,816,5.19%,4.94%,3.98%
3,010019,172,154,122,0.97%,0.80%,0.60%
4,010019,2518,2634,2702,14.25%,13.72%,13.18%
...,...,...,...,...,...,...,...
69779,510046,1,2,2,0.00%,0.01%,0.01%
69780,510046,6,11,9,0.04%,0.06%,0.04%
69781,510046,485,644,552,2.81%,3.39%,2.69%
69782,510046,0,0,0,0.00%,0.00%,0.00%


In [383]:
dataset4['4']=dataset4['4'].str.strip("%").astype(float)/100
dataset4['5']=dataset4['5'].str.strip("%").astype(float)/100
dataset4['6']=dataset4['6'].str.strip("%").astype(float)/100


In [396]:
for col in dataset4:
    dataset4[col]=pd.to_numeric(dataset4[col])
sub_ds1=dataset_close[dataset_close['year_close_combined']==2015]
sub_ds2=dataset_close[dataset_close['year_close_combined']==2016]
sub_ds3=dataset_close[dataset_close['year_close_combined']==2017]
sub_ds=pd.concat([sub_ds1,sub_ds2,sub_ds3])
dataset4['Closure']=dataset4.id.apply(lambda x:1 if x in np.array(sub_ds['id']) else 0)

In [400]:
dataset4

,id,1,2,3,4,5,6,Closure
0,10019,18,31,24,0.0010,0.0016,0.0012,0
1,10019,1062,1480,1540,0.0601,0.0771,0.0752,0
2,10019,917,948,816,0.0519,0.0494,0.0398,0
3,10019,172,154,122,0.0097,0.0080,0.0060,0
4,10019,2518,2634,2702,0.1425,0.1372,0.1318,0
...,...,...,...,...,...,...,...,...
69779,510046,1,2,2,0.0000,0.0001,0.0001,0
69780,510046,6,11,9,0.0004,0.0006,0.0004,0
69781,510046,485,644,552,0.0281,0.0339,0.0269,0
69782,510046,0,0,0,0.0000,0.0000,0.0000,0


In [402]:
dataset4=dataset4.to_numpy().reshape(3172,8*22)

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [403]:
arr1=[i*8 for i in range(22)]
arr2=[i*8+7 for i in range(22)]
arr=np.array([arr1,arr2]).flatten()
arr.sort()

In [404]:
dataset4=np.delete(dataset4,arr[:-1],axis=1)
dataset4.shape

(3172, 133)

In [479]:
dataset1=pd.read_csv('data\hospital_compare\Copy of HQI_HOSP_TimelyEffectiveCare.csv')

In [313]:
dataset1

,Provider ID,Hospital Name,Condition,Measure ID,Measure Name,Score,Sample,Footnote,Measure Start Date,Measure End Date
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,Heart Attack or Chest Pain,AMI_10,Statin at Discharge,98,607,NaN,07/01/2013,06/30/2014
1,010001,SOUTHEAST ALABAMA MEDICAL CENTER,Heart Attack or Chest Pain,AMI_2,Aspirin prescribed at discharge,99,639,NaN,07/01/2013,06/30/2014
2,010001,SOUTHEAST ALABAMA MEDICAL CENTER,Heart Attack or Chest Pain,AMI_7a,Fibrinolytic Therapy Received Within 30 Minute...,Not Available,0,7,07/01/2013,06/30/2014
3,010001,SOUTHEAST ALABAMA MEDICAL CENTER,Heart Attack or Chest Pain,AMI_8a,Primary PCI Received Within 90 Minutes of Hosp...,94,50,NaN,07/01/2013,06/30/2014
4,010001,SOUTHEAST ALABAMA MEDICAL CENTER,Emergency Department,ED_1b,ED1,282,475,2,07/01/2013,06/30/2014
...,...,...,...,...,...,...,...,...,...,...
217710,670092,WALNUT HILL MEDICAL CENTER,Blood Clot Prevention and Treatment,VTE_2,ICU venous thromboembolism prophylaxis,Not Available,Not Available,5,07/01/2013,06/30/2014
217711,670092,WALNUT HILL MEDICAL CENTER,Blood Clot Prevention and Treatment,VTE_3,Anticoagulation overlap therapy,Not Available,Not Available,5,07/01/2013,06/30/2014
217712,670092,WALNUT HILL MEDICAL CENTER,Blood Clot Prevention and Treatment,VTE_4,Unfractionated heparin with dosages/platelet c...,Not Available,Not Available,5,07/01/2013,06/30/2014
217713,670092,WALNUT HILL MEDICAL CENTER,Blood Clot Prevention and Treatment,VTE_5,Warfarin therapy discharge instructions,Not Available,Not Available,5,07/01/2013,06/30/2014


In [314]:
dataset1['Provider ID'].value_counts()

100038    51
340015    51
050045    51
260020    51
340001    51
          ..
05127F    12
08002F    12
06005F    12
50013F    12
05031F    12
Name: Provider ID, Length: 4804, dtype: int64

In [477]:
dataset1=pd.DataFrame(dataset1,columns=['Provider ID','Score','Sample'])

In [478]:
dataset1.replace({'Not Available':np.nan})

,Provider ID,Score,Sample
0,010001,98,607
1,010001,99,639
2,010001,NaN,0
3,010001,94,50
4,010001,282,475
...,...,...,...
217710,670092,NaN,NaN
217711,670092,NaN,NaN
217712,670092,NaN,NaN
217713,670092,NaN,NaN


In [275]:
dataset2=pd.read_csv('data\hospital_compare\Copy of Medicare Hospital Spending by Claim.csv')

In [276]:
dataset2

,Hospital Name,Provider Number,Period,State,Claim Type,Avg Spending Per Episode (Hospital),Avg Spending Per Episode (State),Avg Spending Per Episode (Nation),Percent of Spending (Hospital),Percent of Spending (State),Percent of Spending (Nation),Measure Start Date,Measure End Date
0,SOUTHEAST ALABAMA MEDICAL CENTER,10001,1 to 3 days Prior to Index Hospital Admission,AL,Home Health Agency,$19,$16,$13,0.10%,0.09%,0.07%,1/1/2013,12/31/2013
1,SOUTHEAST ALABAMA MEDICAL CENTER,10001,1 to 3 days Prior to Index Hospital Admission,AL,Hospice,$0,$2,$1,0.00%,0.01%,0.00%,1/1/2013,12/31/2013
2,SOUTHEAST ALABAMA MEDICAL CENTER,10001,1 to 3 days Prior to Index Hospital Admission,AL,Inpatient,$5,$5,$5,0.03%,0.03%,0.02%,1/1/2013,12/31/2013
3,SOUTHEAST ALABAMA MEDICAL CENTER,10001,1 to 3 days Prior to Index Hospital Admission,AL,Outpatient,$138,$76,$113,0.72%,0.42%,0.58%,1/1/2013,12/31/2013
4,SOUTHEAST ALABAMA MEDICAL CENTER,10001,1 to 3 days Prior to Index Hospital Admission,AL,Skilled Nursing Facility,$1,$2,$2,0.01%,0.01%,0.01%,1/1/2013,12/31/2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71055,SCOTT & WHITE HOSPITAL- COLLEGE STATION,670088,1 through 30 days After Discharge from Index H...,TX,Outpatient,$448,$667,$664,2.58%,3.18%,3.39%,1/1/2013,12/31/2013
71056,SCOTT & WHITE HOSPITAL- COLLEGE STATION,670088,1 through 30 days After Discharge from Index H...,TX,Skilled Nursing Facility,"$2,638","$2,729","$3,087",15.19%,13.00%,15.77%,1/1/2013,12/31/2013
71057,SCOTT & WHITE HOSPITAL- COLLEGE STATION,670088,1 through 30 days After Discharge from Index H...,TX,Durable Medical Equipment,$94,$124,$107,0.54%,0.59%,0.55%,1/1/2013,12/31/2013
71058,SCOTT & WHITE HOSPITAL- COLLEGE STATION,670088,1 through 30 days After Discharge from Index H...,TX,Carrier,$755,"$1,211","$1,078",4.35%,5.77%,5.51%,1/1/2013,12/31/2013


In [277]:
dataset2['Period'].value_counts()

1 through 30 days After Discharge from Index Hospital Admission    22610
1 to 3 days Prior to Index Hospital Admission                      22610
During Index Hospital Admission                                    22610
Complete Episode                                                    3230
Name: Period, dtype: int64

In [278]:
dataset2.drop(columns=['Hospital Name','Period','State','Claim Type','Measure Start Date','Measure End Date'],inplace=True)

In [279]:
dataset2['Percent of Spending (Hospital)']=dataset2['Percent of Spending (Hospital)'].str.strip("%").astype(float)/100
dataset2['Percent of Spending (State)']=dataset2['Percent of Spending (State)'].str.strip("%").astype(float)/100
dataset2['Percent of Spending (Nation)']=dataset2['Percent of Spending (Nation)'].str.strip("%").astype(float)/100

In [280]:
dataset2['Avg Spending Per Episode (Hospital)']=dataset2['Avg Spending Per Episode (Hospital)'].str.strip("$")
dataset2['Avg Spending Per Episode (Hospital)']=dataset2['Avg Spending Per Episode (Hospital)'].str.replace(',', '')

In [281]:
dataset2['Avg Spending Per Episode (State)']=dataset2['Avg Spending Per Episode (State)'].str.strip("$")
dataset2['Avg Spending Per Episode (State)']=dataset2['Avg Spending Per Episode (State)'].str.replace(',', '')

In [282]:
dataset2['Avg Spending Per Episode (Nation)']=dataset2['Avg Spending Per Episode (Nation)'].str.strip("$")
dataset2['Avg Spending Per Episode (Nation)']=dataset2['Avg Spending Per Episode (Nation)'].str.replace(',', '')

In [398]:
sub_ds1=dataset_close[dataset_close['year_close_combined']==2014]
sub_ds2=dataset_close[dataset_close['year_close_combined']==2015]
sub_ds3=dataset_close[dataset_close['year_close_combined']==2013]
sub_ds=pd.concat([sub_ds1,sub_ds2,sub_ds3])
dataset2=dataset2.rename(columns={'Provider Number':'id'})
dataset2['Closure']=dataset2.id.apply(lambda x:1 if x in np.array(sub_ds['id']) else 0)

In [399]:
dataset2[dataset2['Closure']==1]['id'].value_counts()

450497    22
200025    22
170010    22
260147    22
40042     22
420054    22
250084    22
450795    22
50435     22
450832    22
670002    22
450884    22
370051    22
180149    22
450005    22
190009    22
110183    22
450188    22
140026    22
450373    22
440174    22
100239    22
180117    22
220067    22
260116    22
Name: id, dtype: int64

In [286]:
for col in dataset2:
    dataset2[col]=pd.to_numeric(dataset2[col])

In [564]:
dataset3=dataset2.to_numpy().reshape(3230,8*22)

In [288]:
dataset3.shape

(3230, 176)

In [289]:
arr1=[i*8 for i in range(22)]
arr2=[i*8+7 for i in range(22)]
arr=np.array([arr1,arr2]).flatten()
arr.sort()
arr[:-1]

array([  0,   7,   8,  15,  16,  23,  24,  31,  32,  39,  40,  47,  48,
        55,  56,  63,  64,  71,  72,  79,  80,  87,  88,  95,  96, 103,
       104, 111, 112, 119, 120, 127, 128, 135, 136, 143, 144, 151, 152,
       159, 160, 167, 168])

In [290]:
dataset3=np.delete(dataset3,arr[:-1],axis=1)

In [291]:
dataset3.shape

(3230, 133)

In [408]:
dataset_=np.concatenate((dataset3,dataset4),axis=0)

In [578]:
dataset_.shape

(6402, 133)

In [579]:
train, test = train_test_split(dataset_, test_size=0.3)

In [580]:
X_train = train[:,:-1]
Y_train = train[:,-1].astype(int)
X_test  = test[:,:-1]
Y_test  = test[:,-1].astype(int)
X_train.shape, Y_train.shape,X_test.shape, Y_test.shape

((4481, 132), (4481,), (1921, 132), (1921,))

In [581]:
logreg = LogisticRegression(solver='newton-cg')
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = logreg.score(X_train, Y_train) * 100
acc_log_t = logreg.score(X_test, Y_test) * 100
print('training score:' +str(acc_log))
print('test score:' +str(acc_log_t))
confusion_matrix(Y_test, Y_pred)

training score:99.64293684445437
test score:99.16710046850599


C:\Users\zys\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:211: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "


array([[1905,    7],
       [   9,    0]], dtype=int64)

In [582]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = random_forest.score(X_train, Y_train) * 100
acc_random_forest_t = random_forest.score(X_test, Y_test) * 100
print('training score:' +str(acc_random_forest))
print('test score:' +str(acc_random_forest_t))
confusion_matrix(Y_test, Y_pred)

training score:100.0
test score:99.53149401353461


array([[1912,    0],
       [   9,    0]], dtype=int64)

In [583]:
xgboost = XGBClassifier()
xgboost.fit(X_train, Y_train)
Y_pred = xgboost.predict(X_test)
xgboost.score(X_train, Y_train)
acc_xgboost = xgboost.score(X_train, Y_train) * 100
acc_xgboost_t = xgboost.score(X_test, Y_test) * 100
print('training score:' +str(acc_xgboost))
print('test score:' +str(acc_xgboost_t))
confusion_matrix(Y_test, Y_pred)

[06:27:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


training score:100.0
test score:99.42738157209786


array([[1910,    2],
       [   9,    0]], dtype=int64)

In [584]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = svc.score(X_train, Y_train) * 100
acc_svc_t = svc.score(X_test, Y_test) * 100
print('training score:' +str(acc_svc))
print('test score:' +str(acc_svc_t))
print(classification_report(Y_test, Y_pred))
confusion_matrix(Y_test, Y_pred)

training score:99.55367105556795
test score:99.53149401353461
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1912
           1       0.00      0.00      0.00         9

    accuracy                           1.00      1921
   macro avg       0.50      0.50      0.50      1921
weighted avg       0.99      1.00      0.99      1921



C:\Users\zys\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[1912,    0],
       [   9,    0]], dtype=int64)

In [585]:
isolation_forest= IsolationForest(contamination=0.003715, behaviour='new')
X_train_= X_train[Y_train==0]
isolation_forest.fit(X_train_)
Y_pred = isolation_forest.predict(X_train)
Y_pred[Y_pred == 1] = 0
Y_pred[Y_pred == -1] = 1
confusion_matrix(Y_train, Y_pred)

C:\Users\zys\Anaconda3\lib\site-packages\sklearn\ensemble\_iforest.py:252: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  warn(


array([[4444,   17],
       [  20,    0]], dtype=int64)

In [586]:
one_class_SVM= OneClassSVM(gamma='scale', nu=0.003715)
X_train_= X_train[Y_train==0]
one_class_SVM.fit(X_train_)
Y_pred = one_class_SVM.predict(X_train)
Y_pred[Y_pred == 1] = 0
Y_pred[Y_pred == -1] = 1
print(classification_report(Y_train, Y_pred))
confusion_matrix(Y_train, Y_pred)

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4461
           1       0.02      0.05      0.02        20

    accuracy                           0.98      4481
   macro avg       0.51      0.52      0.51      4481
weighted avg       0.99      0.98      0.99      4481



array([[4399,   62],
       [  19,    1]], dtype=int64)

In [587]:
scaler = StandardScaler()  

scaler.fit(X_train)  
X_train = scaler.transform(X_train)  

X_test = scaler.transform(X_test)

In [588]:
clf = MLPClassifier(hidden_layer_sizes=(132,132,132))
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc_svc = clf.score(X_train, Y_train) * 100
acc_svc_t = clf.score(X_test, Y_test) * 100
print('training score:' +str(acc_svc))
print('test score:' +str(acc_svc_t))
confusion_matrix(Y_test, Y_pred)

training score:99.70988618611916
test score:99.42738157209786


array([[1910,    2],
       [   9,    0]], dtype=int64)

In [589]:
model = keras.models.Sequential([
    
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [590]:
model.fit(X_train, Y_train, epochs=20,
                    )

Epoch 1/20
141/141 [==============================] - 0s 793us/step - loss: 0.7296 - accuracy: 0.8575
Epoch 2/20
141/141 [==============================] - 0s 764us/step - loss: 0.0549 - accuracy: 0.9955
Epoch 3/20
141/141 [==============================] - 0s 807us/step - loss: 0.0363 - accuracy: 0.9962
Epoch 4/20
141/141 [==============================] - 0s 650us/step - loss: 0.0469 - accuracy: 0.9945
Epoch 5/20
141/141 [==============================] - 0s 731us/step - loss: 0.0359 - accuracy: 0.9961
Epoch 6/20
141/141 [==============================] - 0s 643us/step - loss: 0.0389 - accuracy: 0.9951
Epoch 7/20
141/141 [==============================] - 0s 758us/step - loss: 0.0302 - accuracy: 0.9963
Epoch 8/20
141/141 [==============================] - 0s 743us/step - loss: 0.0273 - accuracy: 0.9965
Epoch 9/20
141/141 [==============================] - 0s 736us/step - loss: 0.0315 - accuracy: 0.9959
Epoch 10/20
141/141 [==============================] - 0s 779us/step - loss: 0.027

In [591]:
model.evaluate(X_test, Y_test)

61/61 [==============================] - 0s 517us/step - loss: 0.0331 - accuracy: 0.9953


[0.033084191381931305, 0.9953149557113647]

In [576]:
Y_pred = model.predict(X_test)
Y_pred.shape

(1921, 10)

In [594]:
tf.math.confusion_matrix(labels=Y_test, predictions=Y_pred)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1910,    2],
       [   9,    0]])>

In [432]:
da=pd.read_csv('data\hospital_compare\Copy of hospital_compare_quality_revised_16.csv')

In [473]:
da

,Unnamed: 0,id,Hospital.Type,Hospital.Ownership,Emergency.Services,ehr,rate,mortality,safety,readmin,patient_exp,effective,timely,efficient_img,Closure
0,1,10001,Acute Care Hospitals,Government - Hospital District or Authority,1,1,3.0,2.0,3.0,2.0,1.0,2.0,2.0,2.0,0
1,2,10005,Acute Care Hospitals,Government - Hospital District or Authority,1,1,3.0,1.0,2.0,3.0,2.0,2.0,3.0,1.0,0
2,3,10006,Acute Care Hospitals,Government - Hospital District or Authority,1,1,2.0,1.0,2.0,2.0,1.0,2.0,3.0,2.0,0
3,4,10007,Acute Care Hospitals,Voluntary non-profit - Private,1,1,2.0,2.0,NaN,1.0,2.0,1.0,3.0,NaN,0
4,5,10008,Acute Care Hospitals,Proprietary,1,1,3.0,2.0,NaN,2.0,NaN,2.0,3.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4802,4803,670114,Acute Care Hospitals,Proprietary,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4803,4804,640001,Acute Care Hospitals,Government - Hospital District or Authority,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4804,4805,650001,Acute Care Hospitals,Government - Local,1,1,3.0,1.0,NaN,2.0,NaN,2.0,NaN,NaN,0
4805,4806,650003,Acute Care Hospitals,Voluntary non-profit - Private,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [475]:
da[da['Closure']==1]['rate'].value_counts(dropna=False)

3.0    5
Name: rate, dtype: int64

In [474]:
da[da['Closure']==0]['rate'].value_counts(dropna=False)

3.0    1769
NaN    1223
4.0     941
2.0     679
1.0     108
5.0      82
Name: rate, dtype: int64

In [470]:
sub_ds1=dataset_close[dataset_close['year_close_combined']==2016]
sub_ds2=dataset_close[dataset_close['year_close_combined']==2017]

sub_ds=pd.concat([sub_ds1,sub_ds2])
da=da.rename(columns={'Provider.ID':'id'})
da['Closure']=da.id.apply(lambda x:1 if x in np.array(sub_ds['id']) else 0)

In [472]:
da['Emergency.Services']=da['Emergency.Services'].replace({'Yes':1,'No':0})

In [ ]:
da['rate']